In [38]:
# %%bash
# rm -rf /home/jordan990301/PCA_Experiments/outputs/juicer
# mkdir /home/jordan990301/PCA_Experiments/outputs/juicer
# mkdir /home/jordan990301/PCA_Experiments/outputs/juicer/origin
# mkdir /home/jordan990301/PCA_Experiments/outputs/juicer/My_PC1
# mkdir /home/jordan990301/PCA_Experiments/outputs/juicer/origin_dropna

# touch /home/jordan990301/PCA_Experiments/outputs/Logs/GM12878_primary_replicate_combined_30.log

# HIC_PATH="/home/jordan990301/PCA_Experiments/data/Rao2014/GM12878/MAPQGE30/GSE63525_GM12878_insitu_primary_replicate_combined_30.hic"
# OUTPUT_PATH="/home/jordan990301/PCA_Experiments/outputs/juicer"
# JUICER_TOOLS_PATH="/home/jordan990301/PCA_Experiments/juicer/juicer_tools.jar"
# NORM="KR"
# TYPE="BP"
# RESOLUTION=1000000

# for CHROM in 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 "X" "Y"
# do
# java -jar $JUICER_TOOLS_PATH pearsons $NORM $HIC_PATH $CHROM $TYPE $RESOLUTION $OUTPUT_PATH/origin/origin_pearson_chrom$CHROM.txt -p
# java -jar $JUICER_TOOLS_PATH eigenvector $NORM $HIC_PATH $CHROM $TYPE $RESOLUTION $OUTPUT_PATH/origin/origin_pc1_chrom$CHROM.txt -p
# done

In [39]:
import numpy as np
import pandas as pd
import hicstraw
from sklearn.decomposition import PCA
from numpy import dot
from numpy.linalg import norm

hic_path = "/home/jordan990301/PCA_Experiments/data/Rao2014/GM12878/MAPQGE30/GSE63525_GM12878_insitu_primary_replicate_combined_30.hic"
input_path = "/home/jordan990301/PCA_Experiments/outputs/juicer/origin"
output_path = "/home/jordan990301/PCA_Experiments/outputs/juicer"
chrom_info = {
    "Cell_Line": [],
    "Chromosome": [],
    "Explained_Variance_PC1": [],
    "Explained_Variance_PC2": [],
    "Explained_Variance_PC3": [],
    "Sum_PC1_PC2": [],
    "Entries_PC1": [],
    "Difference_Count_PC1": [],
    "Cosine_Similarity_PC1": []
}
output_df = pd.DataFrame(chrom_info)

In [40]:
hic = hicstraw.HiCFile(hic_path)
chrom_list= []

for chrom in hic.getChromosomes():
    if (chrom.name != "All" and chrom.name != "MT"):
        chrom_list.append(chrom.name)

for chrom in chrom_list:
    ### Calculated from juicer_tools
    pearson_df = pd.read_table(f"{input_path}/origin_pearson_chrom{chrom}.txt", header=None, sep=" ")
    pearson_df.pop(pearson_df.columns[-1])

    pearson_df = pearson_df.dropna(axis=0, how="all").reset_index(drop=True)
    pearson_df = pearson_df.dropna(axis=1, how="all")

    numpy_pearson_df = pearson_df.values
    pearson_df = pd.DataFrame(numpy_pearson_df)

    #### Calc PCA
    pca = PCA(n_components=len(pearson_df[0]))
    pca.fit(pearson_df)

    My_PC1 = pca.components_[0]
    np.savetxt(f'{output_path}/My_PC1/My_PC1_chrom{chrom}.txt', My_PC1, delimiter='\n', fmt='%1.4f')

    Juicer_PC1 = pd.read_table(f"{input_path}/origin_pc1_chrom{chrom}.txt", header=None, sep=" ")
    Juicer_PC1 = Juicer_PC1.dropna(axis=0, how="all").reset_index(drop=True)
    Juicer_PC1 = Juicer_PC1[0]
    Juicer_PC1 = Juicer_PC1.to_numpy()
    np.savetxt(f'{output_path}/origin_dropna/origin_dropna_PC1_chrom{chrom}.txt', Juicer_PC1, delimiter='\n', fmt='%1.4f')

    cos_sim = dot(My_PC1, Juicer_PC1) / (norm(My_PC1) * norm(Juicer_PC1))

    My_PC1_Pos = My_PC1 > 0
    Juicer_PC1_Pos = Juicer_PC1 > 0

    Difference_Count_PC1 = 0
    for i in range(My_PC1_Pos.shape[0]):
        if(My_PC1_Pos[i] != Juicer_PC1_Pos[i]):
            Difference_Count_PC1 += 1

    if(cos_sim < 0):
        Difference_Count_PC1 = My_PC1_Pos.shape[0] - Difference_Count_PC1

    output_df.loc[len(output_df)] = [
        "GM12878",
        chrom, 
        pca.explained_variance_ratio_[0],
        pca.explained_variance_ratio_[1],
        pca.explained_variance_ratio_[2],
        pca.explained_variance_ratio_[0] + pca.explained_variance_ratio_[1],
        My_PC1_Pos.shape[0],
        Difference_Count_PC1,
        cos_sim
    ] 

display(output_df)



,Cell_Line,Chromosome,Explained_Variance_PC1,Explained_Variance_PC2,Explained_Variance_PC3,Sum_PC1_PC2,Entries_PC1,Difference_Count_PC1,Cosine_Similarity_PC1
0,GM12878,1,0.844930,0.069885,0.040365,0.914815,230,7,-0.984993
1,GM12878,2,0.865367,0.078456,0.039431,0.943822,242,3,0.989763
2,GM12878,3,0.869681,0.058441,0.042336,0.928122,196,6,-0.969955
3,GM12878,4,0.621698,0.355446,0.010419,0.977145,190,8,0.972116
4,GM12878,5,0.671706,0.290950,0.020601,0.962656,179,2,0.994042
5,GM12878,6,0.913586,0.045915,0.026105,0.959502,170,7,0.976556
6,GM12878,7,0.714069,0.174393,0.062453,0.888461,158,7,-0.985678
7,GM12878,8,0.703275,0.243101,0.032962,0.946376,145,6,0.985752
8,GM12878,9,0.689983,0.263110,0.029925,0.953092,125,1,0.996129
9,GM12878,10,0.768365,0.113979,0.046576,0.882344,134,3,-0.994500
